<a href="https://colab.research.google.com/github/mekhi-woods/HiloCATsSN1991bg/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# START-UP
import os
import shutil
if os.path.exists('/content/HiloCATsSN1991bg') == True:
    shutil.rmtree('/content/HiloCATsSN1991bg')
    !git clone https://github.com/mekhi-woods/HiloCATsSN1991bg.git
else:
    !git clone https://github.com/mekhi-woods/HiloCATsSN1991bg.git

!pip install --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple snpy
!pip install requests


In [ ]:
"""
Plotting 91bg like 1a to get an idea of what they look like
"""
import matplotlib.pyplot as plt
import numpy as np

FILTER_WHEEL = ['u', 'g', 'r', 'i', 'B', 'V0']

if __name__ == '__main__':
    KrisciunasPath = "/content/HiloCATsSN1991bg/targetLists/91bglike_justnames.txt"
    KrisciunasNames = np.genfromtxt(KrisciunasPath, dtype=str, delimiter=', ')

    allCPSPhot = "/content/HiloCATsSN1991bg/data/CSPdata/SN_photo.dat"
    allCPSPhotData = np.genfromtxt(allCPSPhot, dtype='str')

    names = allCPSPhotData[:,0]
    filters = allCPSPhotData[:,1]
    time = allCPSPhotData[:,2]
    light = allCPSPhotData[:,3]
    err = allCPSPhotData[:,4]

    plt.figure(figsize=(10,6))
    sigma = 1
    for tar in KrisciunasNames:
        for n in range(len(FILTER_WHEEL)):
            # output_names = names[(names == tar) & (filters == FILTER_WHEEL[n])]
            output_light = light[(names == tar) & (filters == FILTER_WHEEL[n])].astype('float64')
            output_time = time[(names == tar) & (filters == FILTER_WHEEL[n])].astype('float64') + 53000
            output_err = err[(names == tar) & (filters == FILTER_WHEEL[n])].astype('float64')
            plt.errorbar(output_time, output_light, yerr=output_err*sigma, fmt='o', label=FILTER_WHEEL[n])

        plt.title(tar); plt.xlabel('Time [MJD]'); plt.ylabel('Intensity [mag]')
        plt.gca().invert_yaxis()
        plt.legend()
        # plt.savefig('save\\'+str(tar)+'.png')
        plt.show()
        break


In [75]:
"""
SNooPy fitting on CSP data
"""
import matplotlib.pyplot as plt
import numpy as np
import snpy
import os

USE_SAVED = True

def recover_dir():
    if os.path.exists('/content/saved_models') == False:
        os.mkdir('/content/saved_models')
    return

def snpy_fit(filePath, model='max_model', shapeParam='dm15', BandsToFit = ['B','g','r','i'], summarize=True):
    s = snpy.get_sn(filePath)

    # Set model parameters
    s.choose_model(model, stype=shapeParam)
    s.set_restbands() # Auto pick appropriate rest-bands

    # Fit data -- using David configurations
    fitargs = {'mangle':1,'calibration':0} # I don't remember what calibration is
    s.fit(BandsToFit,
          dokcorr=True,
          k_stretch=False,
          reset_kcorrs=True,
          **fitargs)

    # Show results
    if summarize:
        s.summary()
        # for param in s.parameters:
        #     print("{} = {} +/- {}".format(param, s.parameters[param], s.errors[param]))
    return s

def manual_muist_calc(s, M_B = -16.38):
    peakmB = s.get_max(['B'], use_model=True)[1][0]
    return peakmB - M_B

def SNooPy2Fitting(CPSpath, tarNames, model='EBV_model2', shape='st', bands=['B','g','r','i'], output=False):
    tarPaths = []
    for tar in tarNames:
        tarPaths.append(CPSpath+'/SN'+str(tar)+'_snpy.txt')

    SNeDistances = {}
    SNeRedshifts = {}

    for i in range(len(tarPaths)):
        tarName = 'SN'+tarNames[i]
        tarSave = '/content/saved_models/'+tarName+'_'+model+'.snpy'
        print('[ '+str(i+1)+' / '+str(len(tarPaths))+'] Fiting data for '+tarName+'...')

        try:
            # Create/Retrieve Fit
            if os.path.exists(tarSave) and USE_SAVED:
                s_n = snpy.get_sn(tarSave)
            else:
                s_n = snpy_fit(tarPaths[i], model=model, shapeParam=shape, BandsToFit=bands, summarize=output) # Enter snpy fit function
                s_n.save(tarSave)

            # Pull SNooPY distance
            snpy_mu = s_n.get_distmod() # Nab paramaters from SNe objects)

            # Update dictionary/list
            SNeDistances.update({tarName: snpy_mu})
            SNeRedshifts.update({tarName: s_n.z})

            # Print info
            print('Redshift:\t z = '+str(s_n.z))
            print('Distance: \t mu = '+str(round(snpy_mu, 4)))
            print('\n')
        except:
            print('Encountered problem child... skipping\n')
            pass
        plt.clf()

    return SNeDistances, SNeRedshifts

if __name__ == '__main__':
    recover_dir()

    KrisciunasNames = np.genfromtxt("/content/HiloCATsSN1991bg/targetLists/91bglike_justnames.txt", dtype=str, delimiter=', ')

    snpyDistances, snpyRedshifts = SNooPy2Fitting(CPSpath='/content/HiloCATsSN1991bg/data/CSPdata',
                                                  tarNames=KrisciunasNames,
                                                  model='EBV_model2',
                                                  shape='st',
                                                  bands=['B','g','r','i'],
                                                  output=False)



    # burnsData = np.genfromtxt('/content/HiloCATsSN1991bg/targetLists/burns+25table2ext.txt', dtype=str)
    # burnsDistances = {}
    # for tar in np.stack((burnsData[:, 0], burnsData[:, 14]), axis=1):
    #     burnsDistances.update({'SN'+tar[0]: float(tar[1])})

    # # Plot Distance v. Distance
    # print("Ploting differences in distance calculations...")
    # plt.figure(figsize=(8,5))
    # for x in snpyDistances:
    #     try:
    #         plt.scatter(burnsDistances[x], snpyDistances[x], marker='o')
    #         plt.text(burnsDistances[x]+0.05, snpyDistances[x]-0.05, x, fontsize='xx-small')
    #         print(burnsDistances[x], snpyDistances[x])
    #     except KeyError:
    #         print(x, 'not found in Burns+25 Table 2.')
    #         pass
    # plt.title("Burns v. SNooPy Distance"); plt.xlabel('Burns Distance'); plt.ylabel('SNooPy Distance')
    # plt.savefig('burns_v_snpy_dist.png')
    # plt.show()

    # # Plot Residuals
    # print("Ploting residuals...")
    # plt.figure(figsize=(8,5))
    # for x in snpyDistances:
    #     try:
    #         plt.scatter(snpyRedshifts[x], abs(burnsDistances[x]-snpyDistances[x]), marker='o')
    #         plt.text(snpyRedshifts[x], abs(burnsDistances[x]-snpyDistances[x]), x, fontsize='xx-small')
    #         print(snpyRedshifts[x], burnsDistances[x]-snpyDistances[x])
    #     except KeyError:
    #         print(x, 'not found in Burns+25 Table 2.')
    #         pass
    # plt.title("Burns-SNooPy Residuals"); plt.xlabel('Redshift'); plt.ylabel('Burns-SNooPy')
    # plt.savefig('burns-snpy_res.png')
    # plt.show()


[ 1 / 13] Fiting data for SN2005bl...
Redshift:	 z = 0.0241
Distance: 	 mu = 35.0762


[ 2 / 13] Fiting data for SN2005ke...
Redshift:	 z = 0.0049
Distance: 	 mu = 31.3061


[ 3 / 13] Fiting data for SN2006bd...
Encountered problem child... skipping

[ 4 / 13] Fiting data for SN2006gt...
Redshift:	 z = 0.0448
Distance: 	 mu = 36.305


[ 5 / 13] Fiting data for SN2006mr...
Redshift:	 z = 0.0059
Distance: 	 mu = 31.7571


[ 6 / 13] Fiting data for SN2007N...
Redshift:	 z = 0.0129
Distance: 	 mu = 33.7883


[ 7 / 13] Fiting data for SN2007al...
Redshift:	 z = 0.0122
Distance: 	 mu = 33.6794


[ 8 / 13] Fiting data for SN2007ax...


/usr/local/lib/python3.10/dist-packages/snpy/model.py:310: RuntimeWarning: overflow encountered in power
  f = power(10, -0.4*(mod - self.parent.data[band].filter.zp))
/usr/local/lib/python3.10/dist-packages/snpy/model.py:322: RuntimeWarning: invalid value encountered in multiply
  resids_list.append((f - self.parent.data[band].flux)*W)
/usr/local/lib/python3.10/dist-packages/snpy/model.py:313: RuntimeWarning: overflow encountered in power
  cov_f = power(f*err/1.0857,2)


Encountered problem child... skipping

[ 9 / 13] Fiting data for SN2007ba...
Redshift:	 z = 0.0385
Distance: 	 mu = 36.0541


[ 10 / 13] Fiting data for SN2008bd...
Encountered problem child... skipping

[ 11 / 13] Fiting data for SN2008bi...
Redshift:	 z = 0.0134
Distance: 	 mu = 33.8382


[ 12 / 13] Fiting data for SN2008bt...
Redshift:	 z = 0.0154
Distance: 	 mu = 34.1616


[ 13 / 13] Fiting data for SN2009F...
Redshift:	 z = 0.013
Distance: 	 mu = 33.6149


{'SN2005bl': 35.076243726419676, 'SN2005ke': 31.30608249134693, 'SN2006gt': 36.30503452737874, 'SN2006mr': 31.757065254102663, 'SN2007N': 33.78830334230361, 'SN2007al': 33.679398021724815, 'SN2007ba': 36.05405045038354, 'SN2008bi': 33.83824249025314, 'SN2008bt': 34.161591113913516, 'SN2009F': 33.61485276599805}
{'SN2005bl': 0.0241, 'SN2005ke': 0.0049, 'SN2006gt': 0.0448, 'SN2006mr': 0.0059, 'SN2007N': 0.0129, 'SN2007al': 0.0122, 'SN2007ba': 0.0385, 'SN2008bi': 0.0134, 'SN2008bt': 0.0154, 'SN2009F': 0.013}


<Figure size 640x480 with 0 Axes>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import urllib.request
import requests
import os
from requests.auth import HTTPBasicAuth

API_TOKEN = "7f4e1dee8f52cf0c8cdaf55bf29d04bef4810fb4"

ATLAS_PATH = '/content/ATLAS'

def recover_dir():
    if os.path.exists(ATLAS_PATH) == False:
        os.mkdir(ATLAS_PATH)
    return

def main():

        pickle = np.load('tmp.npz', allow_pickle=True)
        data = pickle['data']

        data = requests.post(
            'https://star.pst.qub.ac.uk/sne/atlas4/api/objectlist/',
            headers={'Authorization': f'Token {API_TOKEN}'},
            data={'objectlistid':2}
        ).json()

        np.savez('tmp.npz', data=data)

        count = 0
        for d in data:
            if d['observation_status'] is not None and d['observation_status'].startswith('SN Ia') and '91bg' in d['observation_status']:
                print(d['atlas_designation'],d['observation_status'].replace(' ',''),d['ra'],d['dec'])
                count += 1

                ids = d['id']
                base_url = 'https://star.pst.qub.ac.uk/sne/atlas4/lightcurveforced/1161048951013729300/'
                new_url = base_url.replace('1161048951013729300/',str(ids))
                print(new_url)

                idfile = ATLAS_PATH+'/' + str(ids)+'.txt'
                if os.path.exists(idfile):
                    continue
                urllib.request.urlretrieve(str(new_url), str(idfile))
                print(idfile)

            if count > 300:
                break

def plotting():
    testPath = '/content/ATLAS/1000438990080559900.txt'
    data = np.genfromtxt(testPath, dtype=float, delimiter=',')
    t = data[:, 8]
    uJy = data[:, 24]
    plt.scatter(t, uJy, linewidth=0.1)
    plt.ylim(-250, 230)
    plt.show()



    return

if __name__ == '__main__':
    # recover_dir()
    # print (main())
    plotting()